<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!python Tutorial-Book-Utils/PL_data_loader.py --data FaceMaskDetection

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1pJtohTc9NGNRzHj5IsySR39JIRPfkgD3 

Face Mask Detection.zip download complete!


In [22]:
pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [23]:
import gdown

gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id={'1pJtohTc9NGNRzHj5IsySR39JIRPfkgD3'}",'Face Mask Detection.zip')

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?export=download&confirm=pbef&id=1pJtohTc9NGNRzHj5IsySR39JIRPfkgD3 

